In [1]:
# Construction of dataset

import os, itertools, time, pickle, sys, glob, requests
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
from orderedset import OrderedSet
from copy import deepcopy
%matplotlib inline  

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.construct_mapping_dict()
        
        self.parents_dict = {}
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    
    def construct_mapping_dict(self):
        self.mapping_dict = {self.extract_ID(el, False): self.get_child_node(el, "rdfs:label")[0].firstChild.nodeValue for el in self.root.getElementsByTagName("owl:Class") if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
        
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self, rootpath=False):
        subclasses = self.get_subclasses(rootpath=False)
        return [(b,a,c,d) for (a,b,c,d) in subclasses]
    
    def parse_triples(self, union_flag=0, subclass_of=True, rootpath=False):
        obj_props = [(prop, "Object Property") for prop in self.object_properties]
        data_props = [(prop, "Datatype Property") for prop in self.data_properties]
        props = obj_props + data_props
        all_triples = []
        for prop, prop_type in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop), prop_type) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop), prop_type))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples(rootpath))
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True, rootpath=False):
        return self.parse_triples(union_flag, subclass_of, rootpath)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode, "subclass_of", "Subclass"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    restriction = el.getElementsByTagName("owl:Restriction")
                    if not restriction:
                        continue
                    prop = self.get_child_node(restriction[0], "owl:onProperty")
                    some_vals = self.get_child_node(restriction[0], "owl:someValuesFrom")
                    
                    if not prop or not some_vals:
                        continue
#                     print(self.extract_ID(el), "**", self.extract_ID(some_vals[0]), "**", self.extract_ID(prop[0]))
                    try:
                        if self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                        else:
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                    except:
                        try:
                            if not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                            elif not self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                class_vals = self.get_child_node(some_vals[0], "owl:Class")
                                subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                        except Exception as e:
                            print (e)
                            continue
                else:
                    if self.extract_ID(level1_class[0]):
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of", "Subclass"))
                    else:
#                         level2classes = level1_class[0].getElementsByTagName("owl:Class")
#                         subclass_pairs.extend([(elem, el.parentNode, "subclass_of", "Subclass") for elem in level2classes if self.extract_ID(elem)])
                        continue
        return subclass_pairs
        
    def get_subclasses(self, rootpath=False):
        subclasses = [(self.extract_ID(a, not rootpath), self.extract_ID(b, not rootpath), c, d) for (a,b,c,d) in self.subclasses]
        self.parents_dict = {}
        for (a,b,c,d) in subclasses:
            if c == "subclass_of" and a!="Thing" and b!="Thing":
                if b not in self.parents_dict:
                    self.parents_dict[b] = [a]
                else:
                    self.parents_dict[b].append(a)
        return [el for el in subclasses if el[0] and el[1] and el[2] and el[0]!="Thing" and el[1]!="Thing"]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element, check_coded = True):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))


In [17]:
alignment_folder = "german_datasets/mapping freizeit/"

ontologies_in_alignment = []

# Load reference alignments 
def load_alignments(folder):
    gt = []
    path = [folder + l for l in os.listdir(folder) if l.endswith(".txt")][0]
    mappings = [content.strip() for content in open(path).read().split("--------------------------------------------------------") if content.strip()]
    for mapping in mappings:
        src = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Source: ")][0]
        targ = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Target: ")][0]
        ontologies_in_alignment.append((folder + src, folder + targ))
        src = src.rsplit(".",1)[0].replace(".", "_").lower()
        targ = targ.rsplit(".",1)[0].replace(".", "_").lower()
        lines = [["_".join(row.strip().split(":")[0].replace(",-", "_").split(".")) for row in line.split("-",1)[1].strip().split("<->")]
                 for line in mapping.split("\n") if line.startswith(" -")]
        lines = [[src + "#" + line[0], targ + "#" + line[1]] for line in lines]
        gt.extend(lines)
    return gt

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)


reference_alignments = load_alignments(alignment_folder)


In [3]:
ontologies_in_alignment = [ ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/google.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/web.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl'),
 ('german_datasets_copy/webdirectory/google.owl',
  'german_datasets_copy/webdirectory/web.owl'),
 ('german_datasets_copy/webdirectory/google.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl'),
 ('german_datasets_copy/webdirectory/web.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl')]

In [18]:
# Combinatorial mapping generation
all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology(l[0])
    ont2 = Ontology(l[1])
    
    ent1 = ont1.get_classes()
    ent2 = ont2.get_classes()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()

    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    pre1 = l[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    pre2 = l[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    print (pre1, pre2)
    all_mappings.extend([(pre1 + "#" + el[0], pre2 + "#" + el[1]) for el in mappings])
    

data = {mapping: False for mapping in all_mappings}
reference_alignments = [tuple(alignment) for alignment in reference_alignments]
s = set(all_mappings)
for mapping in set(reference_alignments):
    if mapping in s:
        data[mapping] = True
    else:
        mapping = tuple([el.replace(",-", "_") for el in mapping])
        if mapping in s:
            data[mapping] = True
        else:
            print (mapping)

dmoz_freizeit google_freizeit


In [3]:
ontologies_in_alignment = [('german_datasets_copy/freizeit/dmoz.Freizeit.owl',
  'german_datasets_copy/freizeit/Google.Freizeit.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/google.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/web.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl'),
 ('german_datasets_copy/webdirectory/google.owl',
  'german_datasets_copy/webdirectory/web.owl'),
 ('german_datasets_copy/webdirectory/google.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl'),
 ('german_datasets_copy/webdirectory/web.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl')]
filtered_dict = {}

In [2]:


def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

# Abbrevation resolution preprocessing
def parse(word):
    return " ".join(flatten([el.split("_") for el in camel_case_split(word)]))

abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_") if el])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_") if el])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings([parse(el) for el in keys])))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}


NameError: name 'all_mappings' is not defined

In [4]:

filtered_dict = {}
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

# Abbrevation resolution preprocessing
def parse(word):
    return " ".join(flatten([el.split("_") for el in camel_case_split(word)]))

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)

extracted_elems = []
mapping_ont = {}
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology(ont_name)
    entities = ont.get_classes()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    ont_name = ont_name.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    mapping_ont[ont_name] = ont
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))
inp = []
for word in extracted_elems:
    ont_name = word.split("#")[0]
    elem = word.split("#")[1]
    inp.append(parse(mapping_ont[ont_name].mapping_dict.get(elem, elem)))

print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

extracted_elems = ["<UNK>"] + extracted_elems

embeds = np.array(extractUSEEmbeddings(inp))
embeds = np.array([np.zeros(embeds.shape[1],)] + list(embeds))
# embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_spellchecked)))
embeddings = dict(zip(extracted_elems, embeds))


emb_vals = list(embeddings.values())
emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}


Total number of extracted unique classes and properties from entire RA set:  3032


In [5]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples(rootpath=True)
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append([e2])
            neighbours_dict[e2][2].append([e1])
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append([e2])
            neighbours_dict[e2][3].append([e1])
        elif d == "Subclass":
            neighbours_dict[e2][1].append([e1])
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
#         print ("Done for ", elem)
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()

    
    for entity in neighbours_dict:
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)

# max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
# max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
# neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
#                                 for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
# neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
#                                 for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
# neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
# data_items = np.array(list(data.items()))

# data_shuffled_t = [elem for elem in data_items if elem[1]]
# data_shuffled_f = [elem for elem in data_items if not elem[1]]
# np.random.shuffle(data_items)
# data_shuffled_f = data_shuffled_f[:150000-len(data_shuffled_t)]

# indices = np.random.permutation(len(data_shuffled_t) + len(data_shuffled_f[:130000-len(data_shuffled_t)]))

# data_shuffled = data_shuffled_t + data_shuffled_f
# indices = np.random.permutation(len(data_shuffled))

# indices = np.random.permutation(len(data_shuffled))

# data = OrderedDict(data_items)

# ontologies_in_alignment_rev = [[el.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() for el in ont] for ont in ontologies_in_alignment]

# f = open("Input/data_freizeit.pkl", "wb")
# pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)
# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

german_datasets_copy/freizeit/dmoz.Freizeit.owl
german_datasets_copy/webdirectory/yahoo.small.owl
german_datasets_copy/freizeit/Google.Freizeit.owl
german_datasets_copy/webdirectory/dmoz.owl
german_datasets_copy/webdirectory/google.owl
german_datasets_copy/webdirectory/web.owl


In [6]:
data, _, _, _, neighbours_dicts = pickle.load(open("../Input/data_webdir_bagofnbrs.pkl", "rb"))
f = open("../Input/data_webdir_bagofnbrsv2.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)


In [18]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples(rootpath=True)
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append(e2)
            neighbours_dict[e2][2].append(e1)
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append(e2)
            neighbours_dict[e2][3].append(e1)
        elif d == "Subclass":
            neighbours_dict[e2][1].append(e1)
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
#         print ("Done for ", elem)
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()

    for entity in neighbours_dict:
        neighbours_dict[entity][1] = [neighbours_dict[entity][1]]
        neighbours_dict[entity][2] = [neighbours_dict[entity][2]]
        neighbours_dict[entity][3] = [neighbours_dict[entity][3]]
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)

# max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
# max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
# neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
#                                 for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
# neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
#                                 for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
# neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
# data_items = np.array(list(data.items()))

# data_shuffled_t = [elem for elem in data_items if elem[1]]
# data_shuffled_f = [elem for elem in data_items if not elem[1]]
# np.random.shuffle(data_items)
# data_shuffled_f = data_shuffled_f[:150000-len(data_shuffled_t)]

# indices = np.random.permutation(len(data_shuffled_t) + len(data_shuffled_f[:130000-len(data_shuffled_t)]))

# data_shuffled = data_shuffled_t + data_shuffled_f
# indices = np.random.permutation(len(data_shuffled))

# indices = np.random.permutation(len(data_shuffled))

# data = OrderedDict(data_items)

# ontologies_in_alignment_rev = [[el.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() for el in ont] for ont in ontologies_in_alignment]

# f = open("Input/data_freizeit_bagofnbrs.pkl", "wb")
# pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)
# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

german_datasets_copy/lebensmittel/web.Lebensmittel.owl
german_datasets_copy/webdirectory/yahoo.small.owl
german_datasets_copy/webdirectory/dmoz.owl
german_datasets_copy/lebensmittel/Google.Lebensmittel.owl
german_datasets_copy/webdirectory/google.owl
german_datasets_copy/webdirectory/web.owl


In [77]:
leb_data = pickle.load(open("Input/data_freizeit.pkl", "rb"))[0]
data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts = pickle.load(open("Input/data_webdir.pkl", "rb"))
pickle.dump([leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], open("Input/data_fre_webdir_bagofnbrs.pkl", "wb"))

In [19]:
leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, _ = pickle.load(open("../Input/data_webdir_leb.pkl", "rb"))
pickle.dump([leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], open("../Input/data_webdir_leb_bagofnbrs.pkl", "wb"))
# pickle.dump([pickle.load(open("Input/data_freizeit.pkl", "rb"))[0],
#              pickle.load(open("../Input/data_webdir.pkl", "rb"))[0],
#              emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)

In [17]:
ontologies_in_alignment = [('german_datasets_copy/lebensmittel/Google.Lebensmittel.owl',
  'german_datasets_copy/lebensmittel/web.Lebensmittel.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/google.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/web.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl'),
 ('german_datasets_copy/webdirectory/google.owl',
  'german_datasets_copy/webdirectory/web.owl'),
 ('german_datasets_copy/webdirectory/google.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl'),
 ('german_datasets_copy/webdirectory/web.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl')]

In [ ]:
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts1.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts1.values()]), axis=0)
neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
                                for nbr_type in neighbours_dicts1[elem]] for elem in neighbours_dicts1}
neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
                                for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}


In [27]:
# max_paths
sorted(Counter(flatten([[len(nbr_type) for nbr_type in elem] for elem in pickle.load(open("../Input/data_webdir_fre.pkl", "rb"))[-1].values()])).items())


[(0, 8658),
 (1, 3486),
 (2, 148),
 (3, 99),
 (4, 69),
 (5, 33),
 (6, 32),
 (7, 37),
 (8, 12),
 (9, 11),
 (10, 6),
 (11, 6),
 (12, 8),
 (13, 7),
 (15, 4),
 (16, 3),
 (17, 3),
 (18, 1),
 (19, 2),
 (20, 4),
 (21, 5),
 (22, 3),
 (23, 1),
 (24, 2),
 (26, 3),
 (38, 1),
 (41, 1),
 (42, 1),
 (48, 1),
 (49, 1)]

In [28]:
sorted(Counter(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in pickle.load(open("../Input/data_webdir_fre.pkl", "rb"))[-1].values()])).items())

[(1, 3156),
 (2, 25),
 (3, 426),
 (4, 1243),
 (5, 1054),
 (6, 329),
 (7, 53),
 (8, 26)]

In [ ]:
embedding_dim = np.array(emb_vals).shape[1]
        
name_embedding = nn.Embedding(len(emb_vals), embedding_dim)
name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
name_embedding.weight.requires_grad = False

cosine_sim_layer = nn.CosineSimilarity(dim=1)
output = nn.Linear(2*embedding_dim, 300)
max_pathlen = np.array(list(neighbours_dicts_pathpadded.values())).shape[3]
v = nn.Parameter(torch.DoubleTensor([1/(max_pathlen) for i in range(max_pathlen)]))

nodes = torch.randint(0, len(emb_vals), size=(10,2))
features= torch.randint(0, len(emb_vals), size=(10,2,4,22,6))

nodes = nodes.permute(1,0) # 2 * batch_size
features = features.permute(1,0,2,3,4) # 2 * batch_size * 4 * max_paths * max_pathlen
for i in range(2):
    node_emb = name_embedding(nodes[i]) # batch_size * 512
    feature_emb = name_embedding(features[i]) #  batch_size * 4 * max_paths * max_pathlen * 512

    path_weights = torch.sum(torch.sum(node_emb[:, None, None, None, :] * feature_emb, dim=-1), dim=-1)
    best_path_indices = torch.max(path_weights, dim=2)[1][(..., ) + (None, ) * 3]
    best_path_indices = best_path_indices.expand(-1, -1, -1, max_pathlen,  embedding_dim)
    best_path = torch.gather(feature_emb, 2, best_path_indices).squeeze(2) # batch_size * 4 * max_pathlen * 512
    # Another way: 
    # path_weights = masked_softmax(path_weights)
    # best_path = torch.sum(path_weights.unsqueeze(-1) * feature_emb, dim=2)

    node_weights = torch.sum(node_emb[:, None, None, :] * best_path, dim=-1).unsqueeze(-1)
    torch.matmul(v, node_weights * best_path)

In [ ]:
def masked_softmax(inp):
    inp = inp.double()
    mask = ((inp != 0).double() - 1) * 9999  # for -inf
    return (inp + mask).softmax(dim=-1)
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))

attended_path = torch.bmm(a.reshape(-1, 1, 512), b.reshape(-1, 512, 1))
attended_path = attended_path.reshape(-1, 4, 22, 6)
path_weights = masked_softmax(torch.sum(attended_path, dim=-1))
path_weights.shape
# best_path = torch.sum(path_weights[:, :, :, None, None] * a, dim=2)

In [ ]:
indices = d[(..., ) + (None, ) * 3].expand(-1, -1, -1, 6, 512)
e = torch.gather(b, 2, indices).squeeze(2)

f = torch.sum(a[:,None,None,:] *e,dim=-1).unsqueeze(-1)

g = (f*e)

h = torch.sum((v[None,None,:,None] * g), dim=2)

In [ ]:
c = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,1,6,512))
d = b * c


In [ ]:
e = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)


In [ ]:
fo

In [24]:
len(data)

3318178

In [ ]:
ctimes, dtimes = [], []
for i in range(100):
    a = torch.randn((10,4,22,6,512))
    b = torch.randn((10,4,1,6,512))
    t = time.time()
    c = a * b
    ctimes.append(time.time()-t)
    t = time.time()
    d = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)
    dtimes.append(time.time()-t)

In [ ]:
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))
c = a * torch.sum(b,dim=2).unsqueeze(2)

In [23]:
arr = [('german_datasets_copy/lebensmittel/Google.Lebensmittel.owl',
  'german_datasets_copy/lebensmittel/web.Lebensmittel.owl')]
for ont_pair in arr:
    a, b, c = ont_pair[0], ont_pair[1], ont_pair[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() + "-" + ont_pair[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    !rm -rf $c
    os.mkdir(c)
    java_command = "java -jar logmap-matcher/target/logmap-matcher-4.0.jar MATCHER file:" +  os.path.abspath(a) + \
                     " file:" + os.path.abspath(b) + " " + "/data/Vivek/IBM/IBM-Internship/" + c + "/ false"
    process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
pred_logmap = [[el.split("/")[-1] for el in l.split("\t")[:-1]] for l in open(os.path.abspath(c + "/") + "/logmap2_mappings.tsv",  "r").read().split("\n")[:-1] if not l.startswith("Optional")]


KeyboardInterrupt: 

In [ ]:
gt_mappings = []
for elem in pred_logmap:
    gt_mappings.append(tuple([el.split("#")[0].replace(".v2", "").rsplit(".",1)[0].replace(".", "_").lower() + "#" + el.split("#")[1] for el in elem]))


In [ ]:
data_orig = pickle.load(open("../Input/data_lebensmittel.pkl", "rb"))[0]
data_logmap = {}
for key in data_orig:
    data_logmap[key] = False
s = set(list(data_logmap.keys()))
gt_mappings = [tuple(pair) for pair in gt_mappings]
for mapping in gt_mappings:
    
    if mapping in s:
        data_logmap[mapping] = True
    else:
        mapping = tuple([el.replace(",-", "_") for el in mapping])
        if mapping in s:
            data_logmap[mapping] = True
        else:
            print (mapping)

In [ ]:
all_metrics = []
def return_test_data(data, i):
    data_t = {elem: data[elem] for elem in data if data[elem]}
    data_f = {elem: data[elem] for elem in data if not data[elem]}

    data_t_items = list(data_t.keys())
    data_f_items = list(data_f.keys())

    test_data_t = data_t_items[int((0.2*i)*len(data_t)):int((0.2*i + 0.2)*len(data_t))]
    test_data_f = data_f_items[int((0.2*i)*len(data_f)):int((0.2*i + 0.2)*len(data_f))]
    
    test_data = {}
    for elem in test_data_t:
        test_data[elem] = True
    for elem in test_data_f:
        test_data[elem] = False
    return test_data

for i in range(5):
    test_gt = return_test_data(data_orig, i)
    test_logmap = {elem: data_logmap[elem] for elem in test_gt}
    tp = len([elem for elem in test_gt if test_gt[elem] and test_logmap[elem]])
    fp = len([elem for elem in test_logmap if not test_gt[elem] and test_logmap[elem]])
    fn = len([elem for elem in test_logmap if test_gt[elem] and not test_logmap[elem]])
    
    try:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1score = 2 * precision * recall / (precision + recall)
        f2score = 5 * precision * recall / (4 * precision + recall)
        f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    except Exception as e:
        print (e)
        continue
    all_metrics.append((precision, recall, f1score, f2score, f0_5score))
    

In [ ]:
np.mean(all_metrics, axis=0)

In [ ]:
# AML test
def is_test(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) in test_onto

results = []

for i in list(range(0, ontologies_in_alignment, 3)):
    test_onto = all_ont_pairs[i:i+3]
    for ont_pair in test_onto:
        print (ont_pair)
        a, b, c = ont_pair[0], ont_pair[1], ont_pair[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() + "-" + ont_pair[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
        java_command = "java -jar logmap-matcher/target/logmap-matcher-4.0.jar MATCHER file:" +  os.path.abspath(a)
                     " file:" + os.path.abspath(b) + " " + "/data/Vivek/IBM/IBM-Internship/" + c + "/ false"
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    print (os.listdir("AML-test-results/"))
    pred_aml = load_alignments("AML-test-results/")
    pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
    tp = len([elem for elem in pred_aml if data[elem]])
    fn = len([key for key in gt_mappings if key not in set(pred_aml) and is_test(test_onto, key)])
    fp = len([elem for elem in pred_aml if not data[elem]])

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    _ = [os.remove(f) for f in glob.glob('AML-test-results/*')]
    
print ("Final Results:", np.mean(results, axis=0))

In [ ]:
ontologies_in_alignment = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment][:-1] + [["human", "mouse"]]

In [31]:
output = """Output_att1_13_twostep_webdir_bagofnbrs.txt:Final Results: [0.37303915 0.73183906 0.49381757 0.61332559 0.41345759]
Output_att1_13_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.37570598 0.70258501 0.48932525 0.59812934 0.41414592]
Output_att1_1_twostep_webdir_bagofnbrs.txt:Final Results: [0.35929161 0.74841137 0.48516609 0.6147194  0.40086675]
Output_att1_1_twostep_webdir.txt:Final Results: [0.35267482 0.74256009 0.47802242 0.6078456  0.39398369]
Output_att1_1_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.36455053 0.73329891 0.48649444 0.6092781  0.40512887]
Output_att1_1_twostep_webdir_weighted.txt:Final Results: [0.35188915 0.76351908 0.48124958 0.61808296 0.39424025]
Output_att1_20_twostep_webdir_bagofnbrs.txt:Final Results: [0.36416883 0.72745594 0.48514317 0.60618449 0.40449625]
Output_att1_20_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.354399   0.74109667 0.47908714 0.60779882 0.39554581]
Output_att1_21_twostep_webdir_bagofnbrs.txt:Final Results: [0.36902657 0.73281467 0.49066791 0.61187843 0.40963105]
Output_att1_21_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.38743292 0.69722272 0.49785594 0.60082967 0.42512838]
Output_att13_1_twostep_webdir.txt:Final Results: [0.38043649 0.73378553 0.50088847 0.61859885 0.42090633]
Output_att13_1_twostep_webdir_weighted.txt:Final Results: [0.37303089 0.73233399 0.49381297 0.61345365 0.41344185]
Output_att13_3_twostep_webdir.txt:Final Results: [0.36666945 0.74645778 0.49157854 0.61810869 0.40813697]
Output_att13_3_twostep_webdir_weighted.txt:Final Results: [0.34532464 0.73768204 0.47027683 0.60088609 0.38637681]
Output_att13_4_twostep_webdir.txt:Final Results: [0.39645561 0.68990446 0.50324346 0.60060751 0.4331955 ]
Output_att13_4_twostep_webdir_weighted.txt:Final Results: [0.3738504  0.73037446 0.4942045  0.61295255 0.41416498]
Output_att13_5_twostep_webdir.txt:Final Results: [0.39714232 0.69965818 0.50640304 0.6068528  0.4346242 ]
Output_att13_5_twostep_webdir_weighted.txt:Final Results: [0.35036958 0.7484173  0.47709338 0.60957899 0.39200359]
Output_att13_6_twostep_webdir.txt:Final Results: [0.35663166 0.76841137 0.48706892 0.62412861 0.39940913]
Output_att13_6_twostep_webdir_weighted.txt:Final Results: [0.35832489 0.73231262 0.4807576  0.60529286 0.39892512]
Output_att1_3_twostep_webdir_bagofnbrs.txt:Final Results: [0.37399111 0.72208059 0.49261841 0.60858632 0.41384123]
Output_att1_3_twostep_webdir.txt:Final Results: [0.39723927 0.68453504 0.50259943 0.597883   0.43358185]
Output_att1_3_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.37691497 0.7230562  0.49496412 0.61017995 0.41661186]
Output_att1_3_twostep_webdir_weighted.txt:Final Results: [0.40029411 0.69186042 0.50701738 0.60373644 0.43708027]
Output_att1_4_twostep_webdir_bagofnbrs.txt:Final Results: [0.35794708 0.72793069 0.47964975 0.60290133 0.39835618]
Output_att1_4_twostep_webdir.txt:Final Results: [0.37516837 0.72646965 0.49468759 0.61173453 0.41529315]
Output_att1_4_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.35373656 0.75768204 0.48189172 0.61626415 0.39580988]
Output_att1_4_twostep_webdir_weighted.txt:Final Results: [0.39719197 0.69868376 0.50624844 0.60635853 0.43462025]
Output_att1_5_twostep_webdir_bagofnbrs.txt:Final Results: [0.38730598 0.69429945 0.49699088 0.59901489 0.4247828 ]
Output_att1_5_twostep_webdir.txt:Final Results: [0.39331704 0.70696576 0.50514541 0.60938715 0.43150023]
Output_att1_5_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.37836784 0.73232212 0.49876784 0.6166865  0.41878986]
Output_att1_5_twostep_webdir_weighted.txt:Final Results: [0.3687903  0.73378078 0.49073631 0.61236506 0.40948066]
Output_att1_6_twostep_webdir_bagofnbrs.txt:Final Results: [0.34512376 0.75719898 0.47393478 0.61095433 0.38720446]
Output_att1_6_twostep_webdir.txt:Final Results: [0.3911573  0.71038039 0.50443541 0.6106136  0.4297524 ]
Output_att1_6_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.36253718 0.73574743 0.48537076 0.60967164 0.40333686]
Output_att1_6_twostep_webdir_weighted.txt:Final Results: [0.36548234 0.74694558 0.4906568  0.61775508 0.40700325]
Output_att1_8_twostep_webdir_bagofnbrs.txt:Final Results: [0.33179381 0.78156311 0.46544352 0.61428656 0.37482034]
Output_att1_8_twostep_webdir.txt:Final Results: [0.35276043 0.73719898 0.47653968 0.60440817 0.39360942]
Output_att1_8_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.37394334 0.73477182 0.49545294 0.61567695 0.41459938]
Output_att1_8_twostep_webdir_weighted.txt:Final Results: [0.40728037 0.68015429 0.50933607 0.59963216 0.44275055]
Output_att1_9_twostep_webdir_bagofnbrs.txt:Final Results: [0.36741847 0.72940241 0.48840233 0.60899746 0.40780222]
Output_att1_9_twostep_webdir_weighted_bagofnbrs.txt:Final Results: [0.35434577 0.74402113 0.47963939 0.60932992 0.39565534]
Output_att21_1_twostep_webdir.txt:Final Results: [0.3898797  0.70794612 0.50240486 0.60814083 0.42820225]
Output_att21_1_twostep_webdir_weighted.txt:Final Results: [0.39927236 0.70112397 0.50866448 0.60888963 0.43683737]
Output_att21_3_twostep_webdir.txt:Final Results: [0.38342504 0.72646727 0.50150686 0.61569932 0.42324926]
Output_att21_3_twostep_webdir_weighted.txt:Final Results: [0.37368283 0.7250181  0.49289646 0.60991768 0.41368147]
Output_att2_1_twostep_webdir.txt:Final Results: [0.37513174 0.72208652 0.49351596 0.60908134 0.41492349]
Output_att2_1_twostep_webdir_weighted.txt:Final Results: [0.38208328 0.70697051 0.49590944 0.60403402 0.42069357]
Output_att2_3_twostep_webdir.txt:Final Results: [0.35910756 0.74840662 0.48523346 0.6149197  0.40076784]
Output_att2_3_twostep_webdir_weighted.txt:Final Results: [0.3605811  0.72598065 0.48169035 0.60344859 0.40088614]
Output_att2_4_twostep_webdir.txt:Final Results: [0.38128336 0.71136194 0.49642541 0.60632668 0.42027114]
Output_att2_4_twostep_webdir_weighted.txt:Final Results: [0.37601351 0.72744288 0.49568687 0.61278531 0.41619951]
Output_att2_5_twostep_webdir.txt:Final Results: [0.35842519 0.75182957 0.48514867 0.61612371 0.4002203 ]
Output_att2_5_twostep_webdir_weighted.txt:Final Results: [0.36200579 0.74938698 0.48791297 0.61693183 0.40364826]
Output_att2_6_twostep_webdir.txt:Final Results: [0.3568807  0.75134888 0.48376813 0.6151352  0.39869969]
Output_att2_6_twostep_webdir_weighted.txt:Final Results: [0.37133093 0.72404368 0.49053504 0.60801151 0.41127635]
Output_att2_8_twostep_webdir.txt:Final Results: [0.35303581 0.7464554  0.47874311 0.60958698 0.39441545]
Output_att2_8_twostep_webdir_weighted.txt:Final Results: [0.39283694 0.67869919 0.49720199 0.59198912 0.4287978 ]
Output_att3_1_twostep_webdir.txt:Final Results: [0.3695499  0.72597947 0.48965712 0.60843071 0.40974089]
Output_att3_1_twostep_webdir_weighted.txt:Final Results: [0.41873697 0.66211501 0.51269691 0.5928373  0.45182004]
Output_att3_3_twostep_webdir.txt:Final Results: [0.37572831 0.71965581 0.49349524 0.60804913 0.41535857]
Output_att3_3_twostep_webdir_weighted.txt:Final Results: [0.3655979  0.73428521 0.48778048 0.61058061 0.40627413]
Output_att3_4_twostep_webdir.txt:Final Results: [0.3601435  0.74743457 0.48591262 0.61492109 0.40172186]
Output_att3_4_twostep_webdir_weighted.txt:Final Results: [0.34835836 0.73720135 0.47297352 0.60244156 0.38938231]
Output_att3_5_twostep_webdir.txt:Final Results: [0.34352394 0.77377366 0.47554637 0.61839728 0.38641809]
Output_att3_5_twostep_webdir_weighted.txt:Final Results: [0.37454156 0.7162388  0.49142236 0.60517194 0.41387887]
Output_att3_6_twostep_webdir.txt:Final Results: [0.38746674 0.70746187 0.50033952 0.60674513 0.42586294]
Output_att3_6_twostep_webdir_weighted.txt:Final Results: [0.38856515 0.6972239  0.49888632 0.60148497 0.42625636]
Output_att3_8_twostep_webdir.txt:Final Results: [0.38110049 0.72306569 0.49892844 0.61280473 0.42083766]
Output_att3_8_twostep_webdir_weighted.txt:Final Results: [0.34226252 0.75621625 0.47087739 0.60841546 0.38421425]
Output_att4_1_twostep_webdir.txt:Final Results: [0.3735074  0.72842443 0.49356273 0.61179281 0.41374293]
Output_att4_1_twostep_webdir_weighted.txt:Final Results: [0.38025734 0.72208296 0.49780336 0.6115998  0.41988472]
Output_att4_3_twostep_webdir.txt:Final Results: [0.38692044 0.70063498 0.49802855 0.60230419 0.42477853]
Output_att4_3_twostep_webdir_weighted.txt:Final Results: [0.36014548 0.74012462 0.48424211 0.610794   0.40125617]
Output_att4_4_twostep_webdir.txt:Final Results: [0.39297224 0.69283485 0.50135062 0.60093549 0.4301529 ]
Output_att4_4_twostep_webdir_weighted.txt:Final Results: [0.36570323 0.74256602 0.48990395 0.61547372 0.40696017]
Output_att4_5_twostep_webdir.txt:Final Results: [0.35630205 0.74451724 0.48141757 0.61059751 0.39759285]
Output_att4_5_twostep_webdir_weighted.txt:Final Results: [0.36576894 0.72841849 0.48669344 0.60749999 0.40610431]
Output_att4_6_twostep_webdir.txt:Final Results: [0.35014674 0.74500504 0.47580724 0.60712123 0.39145533]
Output_att4_6_twostep_webdir_weighted.txt:Final Results: [0.34996862 0.74646134 0.47626059 0.60821247 0.39147227]
Output_att4_8_twostep_webdir.txt:Final Results: [0.3262342  0.76596879 0.45753105 0.60325137 0.36853396]
Output_att4_8_twostep_webdir_weighted.txt:Final Results: [0.37480928 0.72209008 0.49343797 0.60915495 0.41468411]
Output_att49_1_twostep_webdir.txt:Final Results: [0.37613216 0.73622574 0.49772792 0.61770665 0.41685363]
Output_att49_1_twostep_webdir_weighted.txt:Final Results: [0.35819953 0.73330722 0.48094879 0.60587813 0.39889291]
Output_att6_1_twostep_webdir.txt:Final Results: [0.34452083 0.76255771 0.47438583 0.61332393 0.38686739]
Output_att6_1_twostep_webdir_weighted.txt:Final Results: [0.37415351 0.72793069 0.49402741 0.61187741 0.41435065]
Output_att6_3_twostep_webdir.txt:Final Results: [0.33882622 0.75961783 0.46851664 0.6083295  0.38100553]
Output_att6_3_twostep_webdir_weighted.txt:Final Results: [0.36854446 0.73329535 0.49009542 0.6115651  0.40909137]
Output_att6_4_twostep_webdir.txt:Final Results: [0.40393381 0.67576286 0.50552223 0.59548654 0.43922901]
Output_att6_4_twostep_webdir_weighted.txt:Final Results: [0.38877211 0.69478013 0.49846148 0.60016527 0.4262844 ]
Output_att6_5_twostep_webdir.txt:Final Results: [0.38340664 0.71720847 0.49914737 0.61020489 0.42255022]
Output_att6_5_twostep_webdir_weighted.txt:Final Results: [0.38523717 0.70891698 0.49895723 0.6066428  0.42385672]
Output_att6_6_twostep_webdir.txt:Final Results: [0.35689774 0.74158329 0.4815686  0.60966785 0.39809641]
Output_att6_6_twostep_webdir_weighted.txt:Final Results: [0.38931034 0.70940597 0.50252362 0.60898691 0.4278466 ]
Output_att6_8_twostep_webdir.txt:Final Results: [0.36340055 0.76158448 0.49186727 0.62448015 0.40578167]
Output_att6_8_twostep_webdir_weighted.txt:Final Results: [0.4170326  0.66357724 0.51208485 0.5933181  0.45046771]
Output_att9_1_twostep_webdir.txt:Final Results: [0.3716757  0.71964275 0.49004319 0.60597474 0.41141342]
Output_att9_1_twostep_webdir_weighted.txt:Final Results: [0.39772596 0.68649694 0.50352902 0.59931366 0.43420752]
Output_att9_3_twostep_webdir.txt:Final Results: [0.3324336  0.77766898 0.46555963 0.61306667 0.37535289]
Output_att9_3_twostep_webdir_weighted.txt:Final Results: [0.40748793 0.68162364 0.50984674 0.60056677 0.44304529]
Output_att9_4_twostep_webdir.txt:Final Results: [0.34791362 0.77327992 0.47959246 0.6209027  0.39081305]
Output_att9_4_twostep_webdir_weighted.txt:Final Results: [0.35891084 0.73428046 0.48157505 0.60650805 0.39957828]
Output_att9_5_twostep_webdir.txt:Final Results: [0.37955661 0.724033   0.49771579 0.61241922 0.41935122]
Output_att9_5_twostep_webdir_weighted.txt:Final Results: [0.37013632 0.7352347  0.49189376 0.61347407 0.41076429]
Output_att9_6_twostep_webdir.txt:Final Results: [0.34679654 0.75817815 0.47560276 0.61237767 0.38890429]
Output_att9_6_twostep_webdir_weighted.txt:Final Results: [0.3526595  0.76060768 0.48164382 0.61737089 0.39494777]
Output_att9_8_twostep_webdir.txt:Final Results: [0.37484207 0.73085989 0.49528201 0.61388206 0.41520732]
Output_att9_8_twostep_webdir_weighted.txt:Final Results: [0.37131077 0.71671711 0.48875237 0.60375373 0.41075463]"""

max([float(l.split(":")[2].strip().split()[2]) for l in output.split("\n")])

0.51269691

In [ ]:
f = open("data_unhas.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment], f)


In [ ]:
def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])
neighbours_dicts = {ont: {el: neighbours_dicts[ont][el][:int(sys.argv[1])] for el in neighbours_dicts[ont]
       if count_non_unk(neighbours_dicts[ont][el]) > int(sys.argv[2])} for ont in neighbours_dicts}

In [ ]:
def add1(elem):
    return elem + 1

v = np.random.random((3,2,3))
v, np.vectorize(add1)([[1,2,3],[1,2,3]])

In [ ]:
neighbours_dicts

In [ ]:
import requests

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

# inp_spellchecked = []
for concept in inp[731:]:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word, response["corrections"][word][0])
        
        inp_spellchecked.append(resolved)
        print (concept, resolved)
    else:
        inp_spellchecked.append(concept)




In [ ]:
querystring = {"text": "technically Organised By"}
response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

In [ ]:
inp_spellchecked[730], inp[731]

In [ ]:
fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_v2.pkl", "rb"))]
fn_baseline, fp_baseline = [dict(el) for el in pickle.load(open("test_best.pkl", "rb"))]
fn_unhas, fp_unhas = [dict(el) for el in pickle.load(open("test_unhas.pkl", "rb"))]
fn_resolved, fp_resolved = [dict(el) for el in pickle.load(open("test_resolved.pkl", "rb"))]

fn_dict, fp_dict = {}, {}
def create_comparison_file(file, idx):
    fn, fp = [dict(el) for el in pickle.load(open(file, "rb"))]
    
    for key in fn:
        if key in fn_dict:
            fn_dict[key][idx] = fn[key]
        else:
            fn_dict[key] = ["N/A" for i in range(4)]
            fn_dict[key][idx] = fn[key]
    
    for key in fp:
        if key in fp_dict:
            fp_dict[key][idx] = fp[key]
        else:
            fp_dict[key] = ["N/A" for i in range(4)]
            fp_dict[key][idx] = fp[key]
    

create_comparison_file("test_best.pkl", 0)
create_comparison_file("test_unhas.pkl", 1)
create_comparison_file("test_v2.pkl", 2)
create_comparison_file("test_resolved.pkl", 3)

open("fn - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fn_dict.items()]))
open("fp - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fp_dict.items()]))

In [ ]:
ontologies_in_alignment = pickle.load(open("data_path.pkl", "rb"))[-1]
ontologies_in_alignment

In [ ]:
d = {('confOf#Organization', 'sigkdd#Organizator'): (1,2,3,4),
 ('iasted#Document', 'sigkdd#Document'): (5,6,78,8)}
[[str(el) for el in flatten(el)] for el in d.items()]

In [ ]:
abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    inp_resolved.append(concept)
inp_resolved

In [ ]:
keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))


In [ ]:
cos_sim(*extractUSEEmbeddings(["Conference Banquet", "Dinner Banquet"]))

In [ ]:
fn

In [ ]:
scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)


In [ ]:
inp_case_handled = []
for concept in inp:
    final_list = []
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", concept):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    case_resolved = " ".join(final_list)
    inp_case_handled.append(case_resolved)
    
inp_case_handled

In [ ]:
Ontology("conference_ontologies/conference.owl").triples

In [ ]:
cos_sim2